## Table of contents

- [Immutability](#Immutability)
- [Composition](#Composition)
- [Higher Order Functions / Functions as Values](#Higher-Order-Functions-/-Functions-as-Values)
- [Referential transparency](#Referential-transparency)
- [Type system](#Type-system)
- [Filtering](#Filtering)
- [Reducing](#Reducing)
- [List comprehensions](#List-comprehensions)
- [Combinining and splitting iterators](#Combinining-and-splitting-iterators)
- [Exercises](#Exercises)
    - [Exercise 1: Pure or impure functions? 🌶️](#Exercise-1:-Pure-or-impure-functions?-🌶️)
    - [Exercise 2: Keeping only multiples of n 🌶️](#Exercise-2:-Keeping-only-multiples-of-n-🌶️)
    - [Exercise 3: Transposing a Matrix 🌶️🌶️](#Exercise-3:-Transposing-a-Matrix-🌶️🌶️)
    - [Exercise 4: Flattening list of lists 🌶️🌶️](#Exercise-4:-Flattening-list-of-lists-🌶️🌶️)
    - [Exercise 5: Counting initials 🌶️🌶️🌶️](#Exercise-5:-Counting-initials-🌶️🌶️🌶️)
    - [Exercise 6: Counting initials frequency 🌶️🌶️🌶️](#Exercise-6:-Counting-initials-frequency-🌶️🌶️🌶️)
    - [Exercise 7: Finding palindromes 🌶️🌶️🌶️](#Exercise-7:-Finding-palindromes-🌶️🌶️🌶️)


In [ ]:
# Important modules
import functools
import itertools
import tutorial.functional_programming as fp


---


## References
Here are some additional references to help you in self-learning. Next to each link, we write if this is a video, text or another type of material

- The [functools reference](https://docs.python.org/3/library/functools.html) from the Python standard library (text)
- The [itertools reference](https://docs.python.org/3/library/itertools.html) from the Python standard library (text)
- [Functional programming howto](https://docs.python.org/3/howto/functional.html) from the Python documentation
- A [good, but old introduction](https://www.youtube.com/watch?v=Ta1bAMOMFOI) of functional programming (video)
- A [very high level](https://www.youtube.com/watch?v=Qa8IfEeBJqk) introduction of functional programming (video, advanced). Interesting, but not Python-specific as it refers to Haskell
- A [general introduction on functional programming](https://www.youtube.com/watch?v=8z_bUIl_uPo). Very worth watching as it uses Python for the examples (video)
- [Principles of functional programming](https://dev.to/jamesrweb/principles-of-functional-programming-4b7c)



## Introduction
Functional programming is an approach to programming where programs are built by composing and running functions that perform a series of transformations on data.
This contrasts with the approach of *imperative programming*, where programs are written as a series of statements which modify the *state* of the computation environment.
Typically, within functional programming, great focus is placed on *composition*, *immutability* and  *purity*.
We are going to define these terms in more detail later.

## Why Functional Programming
Why do we choose functional programming? There are a series of advantages to this approach, namely:
- **Debugging** and **testing** are easy: there are no surprises because every function only does one thing and does not affect any other piece of the program.
- **Parallelization** is trivial: functions are just small *boxes* that take one input and produce an output and do not depend on other parts of the code in an implicit manner through global variables or other shared pieces of state. Thus, it's easy to make several functions run in parallel.

## The basic principles of functional programming
All modern programming languages have *functions* (or methods, procedures, subroutines, subprograms); these are groups of program statements that perform a certain computation. Functions are defined once for the whole program and can be reused at will throughout the program whenever we need to perform the specific computation they are defined for. Using functions, we can split our code in smaller units that are only responsible for a specific *functionality*; this helps us structuring our code in a clean and understandable form. 

The main principles we use in functional programming are:

- *Purity*
- *Functions as values* (*higher order functions*)
- *Immutability of data*
- *Composition*
- *Referential transparency*
- *Type systems*

In the next few sections, we are going to briefly explore these concepts with some examples.

### Pure Functions (Purity)

In functional programming, we try to strive for *purity*, that is we want to define and use functions that only depend on their input, always return the same output for the same inputs and do not have any *side effects*, that is they do not indirectly affect any other part of our program.
You can think of these functions as mathematical functions. 
Other examples of side effects are:
- printing to the program output
- reading or writing files
- generating and using random numbers
To better understand this concept, let us look at the function `my_first_pure_function` we defined below:

In [ ]:
def my_first_pure_function(x: int) -> int: 
    return x + 1

Is this a pure function? - Yes, it is!
Any time we run it, we get the same  result and running the function does not affect any other part of our program:

In [ ]:
x = 1
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")

As a rule of thumb, any function that does not modify variables outside of its *scope* and only uses mathematical functions is a pure function.
Now, consider this function instead:

In [ ]:
x = ["short", "list"] 
def do_something(y: str) -> None:
    x.append(y)

Is this function pure? 

Let's try and run it. 

In [ ]:
print(x)
do_something("a")
print(x)

As you see from the output, the function modified the list `x`. Therefore, this function is not pure. This leads us to the next principle, **immutability**.

#### Quiz of pure functions

In [ ]:
import tutorial.functional_programming as fp
fp.PureFunctions()

### Immutability


When writing programs in functional style, we usually avoid functions like `do_something`.
Instead of modifying existing data (*mutation*), you write functions that transform your data and return new objects.
In the case of the function above, we would rewrite it as follows:

In [ ]:
x = ["short", "list"] 
def do_something_immutable(x: "list[str]", y: str) -> "list[str]":
    return x + [y]

print(x)
print(do_something_immutable(x, "a"))
print(x)

The output shows that `do_something_immutable` does not change `x`.
It also does not reference to `x` outside of the scope but takes it as an argument.
Instead of modfying the original `x` list, it returns a new list.
When we want to keep immutability, this is the style we work with.
If we adopt this style, it is easier to reason about the flow of our program as there are no variable that are being modified *at distance* by other function calls.

### Composition

Another important aspect of functional programming is `composition`. This means that whenever we want to perform multiple operations, we avoid writing intermediate variables, especially so when these are in the global state of the program.

Instead, we design our functions in such a way that one functions output can be directly fed into the next function.

Additionally – but this is not as useful in Python – we use the associative property of function application:

$$
f(g(h)) = f \circ (g \circ h)
$$

where we used $\circ$ to express the composition of functions.

In Python, we can do this using a [`lambda` expression](https://docs.python.org/3/tutorial/controlflow.html?#lambda-expressions):



In [ ]:
def f(x: int) -> int:
    return x + 4

def g(x: int) -> int:
    return x * 3

fg = lambda x: f(g(x))

print(f"{fg(4)=}, {f(g(4))=}")

In this way, we can break down a complex calculation in a series of simpler ones.
This is useful for many things:

- It is easier to find problems in a smaller function
- We do our work only once: every time we need the same action, we just call the same function

### Higher Order Functions / Functions as Values

Another important principle of functional programming is that **functions are values**.
In programming languages (like Python) that support a functional style, we can manipulate functions with the language, we can pass them around in a variable and even use functions as parameters for another function.

As an example, consider the function `function_caller`:

In [ ]:
from typing import Callable, Any  # for type hints

def function_caller(f: Callable[[Any], Any], arg: Any) -> Any:
    print(f"Calling the function {f} with argument {arg}")
    res = f(arg)
    print(f"The result of the call was {res}")
    return res

This function takes a function `f` and its argument `args` as input and return the result of calling the function, while additionally printing a message on the standard output of the program. Let's try this out.
To do so, we define a new function `add_five`:

In [ ]:
def add_five(x: int) -> int:
    return x + 5

Now we can call `function_caller` with `add_five` and another number as arguments:

In [ ]:
function_caller(add_five, 4)

This example was a bit convoluted, but it shows that in Python we can use functions as values and even pass them as arguments to other functions.

This is useful in many cases; a typical example is numerical optimization, where we want to find the parameters of a function that minimize a certain criterion.

Other than these specific use cases, there are some common *higher-order functions*, or functions that take other functions as parameters, that are common in most programming languages.We will look at a few of them in a following section.

### Referential transparency

Another principle of functional programming is called [**referential transparency**](https://en.wikipedia.org/wiki/Referential_transparency). This complex-sounding name simply means that we can replace any expression with its value without changing the behavior of the program.

Consider the following Python code:

In [ ]:
def transparent_function(x: int) -> int:
    return x + 1

y1 = transparent_function(1) + 5

y2 = transparent_function(1) + 5

If `transparent_function` is referentially transparent, we can replace its value in the expression `y1 = transparent_function(1) + 5` and we obtain the same value. In other words, the value of `y` does not change if we execute it again, as shown in the code above: we have two expressions `y1` and `y2` that use the same code and because `x` is referentially transparent, their values are equal.

In this case, this is true because the value of `x()` is simply 5 and is invariant.

However, consider this other example:

In [ ]:
import random   # this is used to generate a random number

def non_transparent_function() -> int:
    return random.randint(0, 10)


# With a lamba we capture the expression x() + 5 in a function,
# we get its value when we call it like a function
y1 = lambda: non_transparent_function() + 5

y2 = lambda: non_transparent_function() + 5

y1() == y2()

In this case, because the function returns a different random number every time we call it, the expression `y1` has a different *value* compared to `y2` even though they both have the same *form*.

## Type system

A typical trait of (modern) functional programming is the heavy use of the *type system*.
This provides a set of rules to classify expressions and values in the language into classes called *types*.
You encountered some basic types in the introduction, for example, `int`, `str` or `float`.

We use type systems to enforce constraints and invariants in our program:

- *Making illegal states unpresentable*
- Making sure we pass the right input to functions
- Checking if we wrote a valid program before it runs

Modern (3.6+) Python allows using *type annotation* to mark the type of values, function parameters and class members.

For example:

```python
def fun(a: int) -> str:
    return f"{a}"
```

means that `fun` takes integer values and returns strings.
Python *does not enforce* this constraint.
There are tools, however, which can be used to check the consistency of types in a Python program.

As an example, consider this function:

In [ ]:
# This function greets a person by name and age
def greet(name: str, age: int):
    print(f"Hello {name}, you are {age} years old")
    
greet("Simone", 34)

Since Python does not check the type of arguments  at compilation time, we can run this (absurd) code:

In [ ]:
greet("Simone", "age")

In a language with stricter typing discipline like Java or C++, we would not even be able to compile our program.
Python lacks this strictness. However, there are tools like [mypy](https://mypy-lang.org/) that can be used to check the type consistency of Python programs without running them.

We can achieve a similar effect using execution-time type checks.
To facilitate our work, we first construct a `Person` class tha encapsulates the attributes of a person.
We will see more about classes in the object-oriented programming tutorial. For the moment, you can see a class as container of data and related functions.

In [ ]:
# This defines a simple class with some helper methods and validations.
# In this way, we are sure that when we pass a person to the `greet_better` function,
# this person will have all needed attributes

class Person:
    name: str
    age: int
    
    # This function is called every time we create a new `Person`
    def __init__(self, name: str, age: int):
        if not isinstance(name, str) or not isinstance(age, int):
            raise ValueError("Invalid input")
        self.age = age
        self.name = name


def greet_better(person: Person):
    if isinstance(person, Person):
        print(f"Hello {person.name}, you are {person.age} years old")
    else:
        raise ValueError("Not a person")

Because of the `isinstance` check in `__init_`, we aren't able to construct a `Person` if `age` is a string (type `str`). Likewise, we cannot call `greet_better` with a value other than a `Person`.


In [ ]:
greet_better(Person("Simone", 34))

try:
    greet_better(Person("Simone", "d"))
except ValueError as e:
    print(f"Ooops, it does not work: {e}")
    
try:
    greet_better(("Simone", "d"))
except ValueError as e:
    print(f"Ooops, it does not work: {e}")

Because this approach is rather cumbersome, there are specific libraries like [attrs](https://www.attrs.org/en/stable/) or [pydantic](https://docs.pydantic.dev/) to perform these types of checks on user-defined data types at runtime.

## Mapping / Iteration 

In this section, we will look at the application of some of the principles stated above in Python.
An important application of functional programming in Python is the manipulation of iterables and lists.

### Using map and filter

Python has the `map` built-in used to apply a given function to all the elements of any `iterable`.

<div class="alert alert-block alert-info">
    <h4><b>Note</b></h4>
    Reminder: an iterable is any object capable of returning its elements <strong>one at a time</strong>
</div>

Let's see an example of a `map` using the `add_five` function:

In [ ]:
map(add_five, [1,2,3,4])

The output is a bit confusing! The reason is that `map` does not return a list but a [map](https://docs.python.org/3/library/functions.html#map) object.
This object is **lazy**, that means that the values are only generated when we access them, for example by iterating over the `map` object. 
Fortunately, we can easily convert this into a list by calling the `list` function:

In [ ]:
list(map(add_five, [1,2,3,4]))

This is equivalent to using a for loop:


In [ ]:
a = []
for i in [1,2,3,4]:
    a.append(add_five(i))
print(a)

However, as you can see, this former style is not very functional because we mutate an existing list `a` by appending the results of calling `add_five` to it.
We will see later that Python offers a syntax called *list comprehension* to express this operation in a more functional style.
`map` is useful when we want to apply the same functions to a list of parameters without writing a `for` loop.
It makes for more concise code while sacrificing some readability.

#### Filtering
Another basic higher order function is `filter`.
As the name says, this function is used to filter an `iterable` using a *predicate function*.
This is a function that takes a value and return `true` or `false`. With `true`, the current element is kept, with `false` it is discarded.
We now try to write a predicate function that only keeps even numbers.

<a name="filter-example"></a>

In [ ]:
def is_even(x: int) -> bool:
    return x % 2 == 0

Now we are ready to try `filter` using `is_even`. Because `filter` returns a `filter` object, we wrap it in `list` to directly see the result as a list:

In [ ]:
list(filter(is_even, [1,2,3,4,5]))

#### Reducing
A third basic HoF is *reduction*.
This is a function that takes a function `f(x, y)` of two arguments and an iterable `it` and applies the function to every element in the iterable *cumulatively* to produce one value.
It works in the following way:

- The first argument of `f`, `x`, is the **current value of the accumulation**. In the beginning, this corresponds to the first element of `it`.
- The second argument of `f`, `y`, is the **current element of the iterable**. In the beginning, this corresponds to the second element of `it`.

Because of this behaviour, this function is handy to compute sums or similar aggregations over a list. In Python, this function is available in the [`functools` module](https://docs.python.org/3/library/functools.html), part of Python's standard library.

As an example of using `reduce`, consider the following snippet:


In [ ]:
import functools


def spy(x: int, y: int) -> int:
    val = x * y
    print(f"x: {x}, y:{y}, result: {val}")
    return val

result = functools.reduce(spy, [1, 2, 3, 4, 5])
print(result)

As you can see, we are computing the cumulative product of `[1, 2, 3, 4, 5]`, that is the *factorial* of 5, or `5!`.
The print statement in `spy` helps us see what goes on inside the function at each step in `reduce`.

### List comprehensions
Many of the operations in the previous section can be performed in a different (some would say more *pythonic*) way using [list comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).
These look like mini for-loops inside square brackets and are very useful to write programs in a functional style while keeping code more readable than using higher order functions.  

For example, if we want to double all integers between 1 and 10 and store the result in a list, we can proceed in the classical imperative way:

In [ ]:
# create an empty list
doubles = []

# every time the loop runs, we add a number to the list
for i in range(10):
    doubles.append(i * 2)

print(f"The doubles are {doubles}")


Or we can use `map` like this:

In [ ]:
doubles = map(lambda x: x * 2, range(10))

print(list(doubles))


Again here we wrapped `doubles` in `list` because `map` is lazy and does not return a list immediately but a `map` object which can be converted into a list using `list`. 
This solution is more elegant and avoids side effects because we do not modify the previously defined `doubles` list, instead we directly produce a new list with the desired result.

Now, we solve the same problem using a list comprehensions:

In [ ]:
doubles = [i * 2 for i in range(10)]
print(doubles)

This looks a lot like the solution above for the for loop, but it is an *expression* instead of a *statement*.
[Expressions](https://www.baeldung.com/cs/expression-vs-statement) in programming are pieces of code that return a value and hence can be composed together to form more complex expressions and used anywhere a value is needed.
In functional style, we prefer working with expressions instead of statements because we try to keep [*referential transparency*](https://en.wikipedia.org/wiki/Referential_transparency).

Compherensions can do more than iterate over a list.
We can use them to filter elements of a list by adding an `if` statement at the end.
For example, consider the example of filtering even numbers  from a list [above](#filter-example); we can rewrite this with list comprehensions like this:

In [ ]:
def is_even(x: int) -> bool:
    return x % 2 == 0

[i for i in range(5) if is_even(i)]

We can also combine (*nest*) list compherensions. This is equivalent to a nested for loop. For example consider this expression:

In [ ]:
[(i,j) for i in range(5) for j in range(5)]

This returns all the combination of numbers between 0 and 4 as a list of pairs.
In the first part of the compherension, we can use any expression instead of just referencing the variable.
For example, we can return a description of the pair `i,j` in text:

In [ ]:
[f"The current pair is {i=}, {j=}" for i in range(5) for j in range(5)]

#### Exercises on iteration/mapping

In [ ]:
%reload_ext tutorial.tests.testsuite

1. Modify the function `solution_filter_even` to filter only the even elements of the list `input_data`. For your solution, you **cannot** use explicit `for` loops.

In [ ]:
%%ipytest

def solution_filter_even(input_data: list[int]) -> list[int]:
    """
    Write your solution here
    """
    pass

2. Modify the function `solution_add_one` to add the integer number `1` to each element of the list `input_data`. For your solution, you **cannot** use explicit `for` loops.

In [ ]:
%%ipytest

def solution_add_one(input_data: list[int]) -> list[int]:
    """
    Write your solution here
    """
    pass

### Combinining and splitting iterators
Sometimes, we need to iterate over multiple lists in a special fashion, for example over two lists in parallel or we want to compute all the combinations of elements of two lists etc.
We can solve many ot these problems with the use of list compherensions, but sometimes there are more elegant solutions by using the [`itertools`](https://docs.python.org/3/library/itertools.html) module, a part of Python's standard library.

Lets see some example of problems we can solve by using these tools.

- Iterating over two lists in parallel: 
    consider the two lists `numbers = [1, 2, 3, 4, 5]` and `words = ["one", "two", "three" , "four", "five"]`. We want to produce a list of strings consisting of the content of `numbers` and `words`, such that each number in `numbers` corresponds to the right word in `words`. We can do this by using the `zip` buil-in function:
`zip` takes a number of `iterables` as an input and returns a list of tuples. The i-th element of this list is a tuple with the i-th element of each iterable, proceeding until the shortes iterable is exhausted. Using `zip`, we solve our problem like this:

In [ ]:
numbers = [1, 2, 3, 4, 5]
words = ["one", "two", "three" , "four", "five"]
[f"The number {i} is called {j}" for i, j in zip(numbers, words)]

-  Producing permutations of elements.
    Suppose we have a list containing the letters `["A", "B", "C", "D", "E"]` and we want to produce all four letter words from them.
    We can use   `itertools.permutations`.
    Once again, we wrap the operation in `list` to obtain a list as output:

In [ ]:
import itertools
letters = ["A", "B", "C"]
list(itertools.permutations(letters, 3))

Another useful trick to know when working with lists is [**unpacking**](https://docs.python.org/3/tutorial/controlflow.html#tut-unpacking-arguments). In Python, we can extract elements from a list using the assignment statement.  For example, if we have a two-element list, we can write

In [ ]:
a, b = [1, 2]

and extract the two elements in the variables `a` and `b`. If you try the same statement with a three-element list you will get an error:

In [ ]:
try:
    a, b = [1, 2, 3]
except ValueError:
    print("It does not work")

However, you can extract *all remaining elements* with the star notation:

In [ ]:
a, *b = [1, 2, 3]

If we use `*` before an argument in a function call, this will unpack the argument and pass each component to the function separately, as you can see in this example:

In [ ]:
def print_variables(a: any, b: any, c: any):
    print(f"a is {a}, b is {b}, c is {c} ")

print_variables(*[1, 2, 3])
print_variables(1, 2, 3)
print_variables(*[[1, 2, 3], [4, 5, 6], [5, 6, 7]])

Be careful, this only works if the list/iterable you unpack is as long as the arguments you want to pass to your function.

## Exercises

In [ ]:
%load_ext tutorial.tests.testsuite

### Exercise 1: Pure or impure functions? 🌶️

For each of the functions below, determine whether they are pure or impure

In [ ]:
def fun1(a: "list[str]") -> None:
    a.append("b")


def fun2(a: int) -> int:
    return a + 2


def fun3(a: "dict[str, str]") -> None:
    a["test"] = "dest"


### Exercise 2: Keeping only multiples of n 🌶️

Given a list `L` of integers, write a function that only keeps the numbers that are multiples of a given constant `k`.

- Example 1: given `nums = [1, 2, 3, 4, 5]`, and `k = 2`, the result must be `[2, 4]`

- Example 2: given `nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] ` and `k = 5`, the result is `[5, 10]`



<div class="alert alert-block alert-info">
    <h4><b>Hints</b></h4>
    <ul>
        <li>
            The operator <code>%</code> (modulo) gives the remainder of division. <code>a % k</code> is 0 when <code>a</code> is a multiple of <code>k</code>. For more information, consult the chapter on basic datatypes. 
        </li>
        <li>
            Write your function in the cell below inside of the <code>solution_exercise1</code> function. The function receives a list <code>l</code> as an input and should return another list
        </li>
    </ul>
<div>




In [ ]:
%%ipytest functional_programming

def solution_exercise2(l: "list[int]", k: int):
    """
    Write your solution here
    """
    pass

### Exercise 3: Transposing a Matrix 🌶️🌶️

Consider a matrix `M` represented row-wise as a list of lists `[[1, 2, 3], [4, 5, 6], [7, 8, 8]]`.
Write a function  that returns the transpose of `M`, the matrix obtained by exchanging rows and columns

- Example 1: given `M=[[1, 0], [0, 1]]`, the result must be `[[1, 0], [0, 1]]`
- Example 2: given `M=[[1, 2, 3], [4, 5, 6], [7, 8, 8]]` the result must be `[1, 4, 7], [2, 5, 8], [3, 6, 8]]`



<div class="alert alert-block alert-info">
    <h4><b>Hints</b></h4>
    <ul>
        <li>
            Write your function in <code>solution_exercise3</code>
        <li>
            The <code>zip</code> function behaves similar to a transposition: <code>zip([1,2], [3,4]) = [(1,3), (3,4)])</code>
        </li>
        <li>
            Write your function in the cell below inside of the <code>solution_exercise3</code> function. The function receives a the "matrix"  as an input <code>m</code> and should return another list
        </li>
    </ul>
<div>


In [ ]:
%%ipytest functional_programming

def solution_exercise3(m: "list[list[int]]") -> "list[list[int]]":
    """
    Write your solution here
    """
    pass


### Exercise 4: Flattening list of lists 🌶️🌶️

Imagine we receive a list of lists `l` like `[[1, 2], [3, 4]]`. Write a function that converts this list into a `flat` list like `[1, 2, 3, 4]`. 

To write your function, just complete the code in `solution_exercise4`


<div class="alert alert-block alert-info">
    <h4><b>Hints</b></h4>
    <ul>
        <li>
            This is a good exercise to use <code>functools.reduce</code> 
        </li>
        <li>
            Write your function in the cell below inside of the <code>solution_exercise4</code> function. The function receives a list <code>l</code> as an input and should return another list
        </li>
    </ul>
<div>

In [ ]:
%%ipytest functional_programming

def solution_exercise4(l: "list[list[any]]") -> "list[any]":
    """
    Write your solution here
    """
    pass



### Exercise 5: Counting initials 🌶️🌶️🌶️

Given a list `w` of English words, write a function that counts how many words begin with each letter of the English alphabet and returns the result as an **alphabetically sorted** list of tuples `(letter, count)`. We do not distinguish between uppercase and lowercase letters ("A" and "a" are considered the same).  


<div class="alert alert-block alert-info">
    <h4><b>Hints</b></h4>
    <ul>
        <li>
            Consider the functions <code>sorted</code> and <code>itertools.groupby</code> from the Python standard library.
        </li>
        <li>
            Write your function in the cell below inside of the <code>solution_exercise5</code> function. The function receives a list <code>w</code> as an input and should return another list
        </li>
        <li>
            To ensure consistent capitalization you can use the <code>lower()</code> method of <code>str</code>
        </li>
    </ul>
<div>



In [ ]:
%%ipytest functional_programming

def solution_exercise5(w: list[str]) -> list[(str, int)]:
    """
    Write your solution here
    """
    pass



### Exercise 6: Counting initials frequency 🌶️🌶️🌶️
If you could solve the previous exercise, you now have a list `l` of the form `[('a', 20), ('b', 30), ...]`.
If you cannot, do not worry: you will receive the correct input automatically as `l` inside the function `solution_exercise6`


Write a function that computes the *relative frequency* of each letter in the list `l`. 


<div class="alert alert-block alert-info">
    <h4><b>Hints</b></h4>
    <ul>
        <li>
            The relative frequence of a value <code>a</code> in a list is simply the number of time it appears in that
            list over the total lenght of the list
        </li>
        <li>
            Write your function in the cell below inside of the <code>solution_exercise6</code> function. The function receives a list <code>l</code> as an input and should return another list
        </li>
    </ul>
<div>

In [ ]:
%%ipytest functional_programming

def solution_exercise6(l: "list[(str, int)]") -> "list[(str, float)]":
    pass

### Exercise 7: Finding palindromes 🌶️🌶️🌶️
Consider again the `words` list of strings. Write a function  that returns the list of all *palindromes*. A *palindrome* is a word (any string longer than 1) that reads the same left-to-right and right-to-left.

For example:
- rotator
- wow
- noon
- radar




<div class="alert alert-block alert-info">
    <h4><b>Hints</b></h4>
    <ul>
        <li>
            A single character does not count as a palindrome.
        </li>
        <li>
            The words are available as the input <code>words</code> to <code>solution_exercise7</code>.
        </li>
    </ul>
<div>


In [ ]:
%%ipytest functional_programming

def solution_exercise7(words: "list[str]") -> "list[str]":
    pass